## The Wonderful World of ML - Session 3 Assignment (Linear Regression)

In addition to the references provided in session 2, I've uploading all the video and course notes from the [**Regression Models**](https://www.coursera.org/learn/regression-models) class from the Johns Hopkins Data Science Specialization on coursera to our meetup repo [here](https://github.com/focods/WonderfulML/tree/master/docs/JHU_DSS_RegMods).  This weeks assignment is adapted from an assignment given in this class.

If you haven't done so by now, install jupyter notebook and configure it with an R kernel if you are an R user.  If you are Python user, your Anaconda install will have Python configured out of the box.  For further details, refer to the top of the session 2 notebooks.

### Only one problem for this session, but it's a good one...

2) What was the cost function Sondra mentioned that is used for linear regression?

The data for this assignment can be found in our meetup repo:

[https://raw.githubusercontent.com/focods/WonderfulML/master/data/mtcars.csv](https://raw.githubusercontent.com/focods/WonderfulML/master/data/mtcars.csv)

**Here is the scenario:**  *You work for Motor Trend, a magazine about the automobile industry. Looking at a data set of a collection of cars, they are interested in exploring the relationship between a set of variables and miles per gallon (MPG) (outcome). They are particularly interested in the following two questions:*

1. Is an automatic or manual transmission better for MPG?  
2. Quantify the MPG difference between automatic and manual transmissions.

Here is some code to get you started.

Coming soon!

$$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\Yv}{\mathbf{Y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\betav}{\mathbf{\beta}}
\newcommand{\gv}{\mathbf{g}}
\newcommand{\Hv}{\mathbf{H}}
\newcommand{\dv}{\mathbf{d}}
\newcommand{\Vv}{\mathbf{V}}
\newcommand{\vv}{\mathbf{v}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\Sv}{\mathbf{S}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\Zv}{\mathbf{Z}}
\newcommand{\Norm}{\mathcal{N}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}
\newcommand{\dimensionbar}[1]{\underset{#1}{\operatorname{|}}}
\newcommand{\dimensionbar}[1]{\underset{#1}{\operatorname{|}}}
\newcommand{\grad}{\mathbf{\nabla}}
\newcommand{\ebx}[1]{e^{\wv_{#1}^T \xv_n}}
\newcommand{\eby}[1]{e^{y_{n,#1}}}
\newcommand{\Tiv}{\mathbf{Ti}}
\newcommand{\Fv}{\mathbf{F}}
\newcommand{\ones}[1]{\mathbf{1}_{#1}}
$$

## The Wonderful World of ML - Session 3 Discussion: Logistic Regression

<img src="https://raw.githubusercontent.com/MichaelSzczepaniak/WonderfulML/master/docs/graphics/child_on_logistic_curve_70per.jpg">

Logistic regression provides a straightforward way to model the probability that one of two mutually exclusive outcomes will occur. This is another way of saying that this is a technique used for **binary classification**. Because this is one of the simplest and most popular classifiers out there, this should be in every machine learning practicioners toolbox.  It also serves as a good starting point for more sophisticated classification techniques such as LDA and QDA which we'll cover in the next session.

What are some examples of binary classification?
+ Will a team win or loss a game?
+ Will a customer buy or not buy?
+ Will a person survive 90 days after a myocardial infarction (heart attack)?
+ Will a passenger survive the Titanic disaster? (kaggle competition)

### What is logistic regression modeling?

At first look, it may seem odd to call this classifier logistic *regression* until you realize that this technique is modeling the underlying conditional probability of an event occuring given one or more predictors.  In other words, we are fitting a function to $p(C=k\,|\, \xv_n)$ where $C=\text{class of }
n^\text{th}\text{ sample}$ which is binary meaning that k can only be 0 or 1.  The term $\xv_n$ is a vector of predictors for the $n^{th}$ sample.

### How is this model constructed?

A natural way to think about the modeling a probablity is to divide the count of an event by the total number of events.  If we model the count or frequency of a given event as a function of our predictors $\xv$ and some parameters $\wv$, then we could model the probablity by dividing $f(\xv;\wv_k)$ for a particular class $k$ by the total count in all classes which can be expressed as:

$$
    \begin{align*}
      p(C=k|\xv) = \frac{f(\xv;\wv_k)}{\sum_{m=1}^K f(\xv;\wv_m)} = g_k(\xv)
    \end{align*}
$$

The above ensures that $p(C=k|\xv)$ will be between 0 and 1 if $f(\xv;\wv_k) \gt 0$, but we have another contraint which is:

$$
      \begin{align*}
      1  = \sum_{k=1}^K p_k(C=k|\xv) = \sum_{k=1}^K g_k(\xv)
      \end{align*}
$$



### How do we find the best parameters for the model?

### How do we handle discrete and continuous variables in our model?

### What tools are available in Python to do logistic regression?